In [207]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report as report, recall_score, f1_score, precision_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.stats import binom
from scipy.stats import norm

In [208]:
df = pd.read_excel(r'Data.xls', header=1, usecols='B:M')
df.shape

(5804, 12)

In [209]:
df = df[df.CUSTOMER_ID.notnull()]
df.isnull().sum()
#nie ma już żadnych NULL / NaN

CUSTOMER_ID              0
ASSESSMENT_YEAR          0
PRODUCT_DEMAND           0
OWNERS_MANAGEMENT        0
ACCESS_CREDIT            0
PROFITABILITY            0
SHORT_TERM_LIQUIDITY     0
MEDIUM_TERM_LIQUIDITY    0
GROUP_FLAG               0
TURNOVER                 0
INDUSTRY                 0
DEFAULT_FLAG             0
dtype: int64

In [210]:
df.dtypes
#formaty zmiennych

CUSTOMER_ID              float64
ASSESSMENT_YEAR          float64
PRODUCT_DEMAND           float64
OWNERS_MANAGEMENT        float64
ACCESS_CREDIT            float64
PROFITABILITY            float64
SHORT_TERM_LIQUIDITY     float64
MEDIUM_TERM_LIQUIDITY    float64
GROUP_FLAG               float64
TURNOVER                 float64
INDUSTRY                  object
DEFAULT_FLAG             float64
dtype: object

In [211]:
df = df.drop_duplicates()
df.shape
#usunięte zduplikowane wiersze

(5741, 12)

In [212]:
df[df.duplicated('CUSTOMER_ID')==True].shape
#są klienci którzy wsytępują kilkukrotnie, ale 
#z różnymi oceniami i/lub flagami default

(1112, 12)

In [213]:
#wyrzucamy rok, ID
df = df.drop(['CUSTOMER_ID','ASSESSMENT_YEAR'],axis=1)

In [214]:
df.head()

,PRODUCT_DEMAND,OWNERS_MANAGEMENT,ACCESS_CREDIT,PROFITABILITY,SHORT_TERM_LIQUIDITY,MEDIUM_TERM_LIQUIDITY,GROUP_FLAG,TURNOVER,INDUSTRY,DEFAULT_FLAG
0,55.0,55.0,50.0,50.0,50.0,48.0,0.0,6000158.40,Trade,0.0
1,85.0,80.0,85.0,70.0,80.0,80.0,0.0,6000404.88,Property and Construction Sectors,0.0
2,50.0,40.0,40.0,40.0,40.0,50.0,1.0,6001002.00,"Transport, Storage and Communications Infrastr...",0.0
3,60.0,65.0,60.0,65.0,65.0,63.0,0.0,6002305.68,Property and Construction Sectors,0.0
4,58.0,51.0,54.0,55.0,60.0,58.0,0.0,6002603.91,Manufacturing,0.0


In [215]:
#variables = ['PRODUCT_DEMAND', 'OWNERS_MANAGEMENT', 'ACCESS_CREDIT', 'PROFITABILITY', 'SHORT_TERM_LIQUIDITY', 'MEDIUM_TERM_LIQUIDITY', 'GROUP_FLAG', 'TURNOVER']
explained_variables = ['DEFAULT_FLAG']
data = pd.get_dummies(df,columns=['GROUP_FLAG','INDUSTRY'],prefix=['GROUP_FLAG','INDUSTRY']).drop(['DEFAULT_FLAG'], axis=1)
results = df['DEFAULT_FLAG']
#data = df.loc[df['GROUP_FLAG'].isin([1.])][variables]

In [216]:
data.tail()

,PRODUCT_DEMAND,OWNERS_MANAGEMENT,ACCESS_CREDIT,PROFITABILITY,SHORT_TERM_LIQUIDITY,MEDIUM_TERM_LIQUIDITY,TURNOVER,GROUP_FLAG_0.0,GROUP_FLAG_1.0,"INDUSTRY_Agriculture, Livestock and Fisheries","INDUSTRY_Electricity, Gas and Water",INDUSTRY_Extractive Industries,INDUSTRY_Hotels and Leisure,INDUSTRY_Manufacturing,INDUSTRY_Office Machinery and Computer Industries,INDUSTRY_Other,INDUSTRY_Property and Construction Sectors,INDUSTRY_Trade,"INDUSTRY_Transport, Storage and Communications Infrastructure"
5796,60.0,62.0,60.0,60.0,60.0,60.0,69829351.97,0,1,0,0,0,0,0,0,0,0,1,0
5797,60.0,65.0,55.0,60.0,55.0,55.0,69883750.26,0,1,0,0,0,0,1,0,0,0,0,0
5798,65.0,65.0,68.0,62.0,65.0,70.0,69944866.89,0,1,0,0,0,0,0,0,0,1,0,0
5799,60.0,60.0,58.0,57.0,58.0,55.0,69956000.00,0,1,0,0,0,0,0,0,1,0,0,0
5800,53.0,50.0,45.0,40.0,45.0,58.0,69969828.17,0,1,0,0,0,0,1,0,0,0,0,0


In [217]:
X = data.to_numpy()
y = results.to_numpy()

poly = PolynomialFeatures(1)
X_poly = poly.fit_transform(X)

scaler = StandardScaler()
scaler.fit(X_poly)
X_scaled = scaler.transform(X_poly)

X_final = X_scaled

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y,test_size=.1)

model = LogisticRegression(class_weight='balanced',max_iter=10**3).fit(X_train,y_train)

In [219]:
print(model.score(X_train,y_train),model.score(X_test,y_test)) #accuracy na train i na test

0.9024390243902439 0.9078260869565218


In [220]:
y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(report(y_test,y_pred))

[[466  52]
 [  1  56]]
              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95       518
         1.0       0.52      0.98      0.68        57

    accuracy                           0.91       575
   macro avg       0.76      0.94      0.81       575
weighted avg       0.95      0.91      0.92       575



In [222]:
#sprawdzamy, dla których zmiennych współczynniki są największe
#jest to poprawna metoda sprawdzania ważności zmiennych niezależnych
#ponieważ przed wytrenowaniem rergesji przeskalowaliśmy wszystkie wielkości tak, by miały ten sam rozkład

keys = list(data.keys()).copy()
keys.insert(0,'0')

L = len(keys)
labels = [keys[i]+'_'+keys[j] for i in range(L) for j in range(i,L)]
 
coefs = dict(zip(labels,model.coef_[0]))
#print(model.coef_)
sorted(coefs.items(),key=lambda item: abs(item[1]))

[('0_0', 0.0),
 ('0_INDUSTRY_Electricity, Gas and Water ', 0.0031242434306664713),
 ('0_TURNOVER', 0.009988786863926263),
 ('0_INDUSTRY_Agriculture, Livestock and Fisheries', -0.04194289292060322),
 ('0_INDUSTRY_Manufacturing', -0.04486396109694689),
 ('0_INDUSTRY_Hotels and Leisure', -0.04534827397844909),
 ('0_INDUSTRY_Extractive Industries ', 0.06149631576272823),
 ('0_INDUSTRY_Trade', -0.08451003232643943),
 ('0_INDUSTRY_Transport, Storage and Communications Infrastructure',
  -0.11130269339422916),
 ('0_GROUP_FLAG_0.0', 0.12207702469603905),
 ('0_GROUP_FLAG_1.0', -0.12207702469603905),
 ('0_INDUSTRY_Office Machinery and Computer Industries', 0.12384770946809422),
 ('0_MEDIUM_TERM_LIQUIDITY', 0.14324042698811254),
 ('0_ACCESS_CREDIT', -0.18303897754840534),
 ('0_INDUSTRY_Other', -0.1907971389868644),
 ('0_SHORT_TERM_LIQUIDITY', -0.21990367128454816),
 ('0_INDUSTRY_Property and Construction Sectors', 0.2451138165721563),
 ('0_PROFITABILITY', -0.3668400222568756),
 ('0_OWNERS_MANAGEM

In [224]:
#sprawdzimy teraz, jak wyglądają scores dla kolejnych zmiennych

L = len(X_final[0])
for i in range(L-1):
    print(keys[i+1])
    X_slice = X_final[:,i+1].reshape(-1,1)
    slice_model = LogisticRegression(class_weight='balanced',max_iter=10**3,tol=.1**8).fit(X_slice,y)
    y_pred_slice = slice_model.predict(X_slice)
    #print(recall_score(y_true=y,y_pred=y_pred_slice))
    print(precision_score(y_true=y,y_pred=y_pred_slice),recall_score(y_true=y,y_pred=y_pred_slice),f1_score(y_true=y,y_pred=y_pred_slice))

PRODUCT_DEMAND
0.46684350132625996 1.0 0.6365280289330922
OWNERS_MANAGEMENT
0.32117367168913563 0.7670454545454546 0.4527669088876468
ACCESS_CREDIT
0.2720045428733674 0.9071969696969697 0.41852337265181294
PROFITABILITY
0.20447696943607405 0.8996212121212122 0.3332164152928797
SHORT_TERM_LIQUIDITY
0.24693986162852583 0.8787878787878788 0.38554216867469887
MEDIUM_TERM_LIQUIDITY
0.22211253701875616 0.8522727272727273 0.3523884103367267
TURNOVER
0.09978249048395867 0.6950757575757576 0.17451260104612457
GROUP_FLAG_0.0
0.09246901811248809 0.36742424242424243 0.14775323686214775
GROUP_FLAG_1.0
0.09246901811248809 0.36742424242424243 0.14775323686214775
INDUSTRY_Agriculture, Livestock and Fisheries
0.09231878831590336 0.9696969696969697 0.16858742179782682
INDUSTRY_Electricity, Gas and Water 
0.09206349206349207 0.9886363636363636 0.16844143272023235
INDUSTRY_Extractive Industries 
0.1686746987951807 0.026515151515151516 0.04582651391162029
INDUSTRY_Hotels and Leisure
0.09267431597528684 0.9

In [ ]:
#z powyższych analiz wynika, że zmiennymi które mają największe znaczenie są przede wszystkim 
#Product Demand oraz w mniejszym stopniu Owners_Management

### Multivariate analysis

TESTY STATYSTYCZNE

In [51]:
column1 =pd.DataFrame(y_test.reshape(-1,1), columns=["Real"])
column2 =pd.DataFrame(y_pred.reshape(-1,1), columns=["Predicted"])
column3 = pd.DataFrame(model.predict_proba(X_test)[:,1].reshape(-1,1), columns=["Probability"])

In [52]:
Data = pd.concat([column1,column2,column3], axis=1)

In [53]:
Data=Data.sort_values(by=["Probability"])

In [55]:
Data=Data.reset_index(drop=True)
Data=Data.round(4)
Data

,Real,Predicted,Probability
0,0.0,0.0,0.0000
1,0.0,0.0,0.0000
2,0.0,0.0,0.0000
3,0.0,0.0,0.0000
4,0.0,0.0,0.0000
...,...,...,...
570,0.0,1.0,0.9998
571,1.0,1.0,0.9998
572,1.0,1.0,0.9999
573,1.0,1.0,0.9999


TESTS

HOSMAR-LEMESHOW AND BRIER STAT

In [56]:
#Funkcja, przyjmuje tabelę o kolumnach "Real", "Predicted" i "Probability" posortowanej według tej ostatniej, oraz listę wartości w jakich ma posortować

def Hosmar_Lemeshow_Brier(DF):
    Data_grouped=pd.DataFrame(columns=["Number of Goods", "Number of Bads", "Total Number", "Realized default rate", "Probability of default"])
    indices = list(np.array_split(range(DF.shape[0]), 10))
    for i in indices:
        Data_temp=DF.iloc[i]
        row=[len(Data_temp)-sum(Data_temp["Real"]),sum(Data_temp["Real"]),len(Data_temp),np.mean(Data_temp["Real"]),np.mean(Data_temp["Probability"])]
        Data_grouped.loc[len(Data_grouped)]=row
    HL =[]
    for i in range(10):
        p_obs_tym=Data_grouped["Realized default rate"][i]
        p_pred_tym=Data_grouped["Probability of default"][i]
        Ng_tym=Data_grouped["Total Number"][i]
        if p_pred_tym >0 and p_pred_tym <1: 
            HL.append(Ng_tym*((p_obs_tym-p_pred_tym)**2)/(p_pred_tym*(1-p_pred_tym)))
        else:
            HL.append(0) 
    Data_grouped["HL stat"]=HL
    p_obs=Data_grouped["Realized default rate"]
    p_pred=Data_grouped["Probability of default"]
    Ng=Data_grouped["Total Number"]
    Data_grouped["Brier Score"] =Ng*(p_obs*(1-p_obs)+(p_pred-p_obs)**2)
    Data_grouped.loc[len(Data_grouped)]=[len(Data)-sum(Data["Real"]),sum(Data["Real"]),len(Data),np.mean(Data["Real"]),np.mean(Data["Probability"]),sum(Data_grouped["HL stat"]),sum(Data_grouped["Brier Score"])/len(Data)]
    lista=list(range(1,11))
    lista.append("All classes")
    Data_grouped.insert(0,"Rating class", lista)
    Data_grouped=Data_grouped.set_index("Rating class")
    Data_grouped=Data_grouped.round(4)
    return Data_grouped

In [57]:
Data1=Hosmar_Lemeshow_Brier(Data)
Data1

,Number of Goods,Number of Bads,Total Number,Realized default rate,Probability of default,HL stat,Brier Score
Rating class,,,,,,,
1,58.0,0.0,58.0,0.0000,0.0000,0.0000,0.0000
2,58.0,0.0,58.0,0.0000,0.0000,0.0000,0.0000
3,58.0,0.0,58.0,0.0000,0.0001,0.0067,0.0000
4,58.0,0.0,58.0,0.0000,0.0006,0.0332,0.0000
5,58.0,0.0,58.0,0.0000,0.0020,0.1134,0.0002
6,57.0,0.0,57.0,0.0000,0.0166,0.9631,0.0157
7,57.0,0.0,57.0,0.0000,0.0692,4.2401,0.2732
8,52.0,5.0,57.0,0.0877,0.2911,11.4278,6.9198
9,31.0,26.0,57.0,0.4561,0.7716,32.1780,19.8117


BINOMIAL AND NORMAL TEST

In [58]:
# Funkcja przyjmuje tabelę, która w której są dane podziału na N grup, w każdej grupie znana jest liczba przypadków oraz probability of default

def Binomial_Test(DF):
    DATA_bin=pd.DataFrame(columns=["5.0%","95.0%","1.0%","99.0%","0.01%","99.9%"])
    def qb(q,N,p):
        return binom.ppf(q,N,p)/N
    for i in range(len(DF)):
        N_temp=DF["Total Number"][i+1]
        prob_tym=DF["Probability of default"][i+1]
        DATA_bin.loc[len(DATA_bin)]=[qb(0.025,N_temp,prob_tym),qb(0.975,N_temp,prob_tym),qb(1-(1+0.98)/2,N_temp,prob_tym),qb((1+0.98)/2,N_temp,prob_tym),qb(1-(1+0.998)/2,N_temp,prob_tym),qb((1+0.998)/2,N_temp,prob_tym)]
    lista=list(range(1,len(DF)+1))
    DATA_bin.insert(0,"Rating class", lista)
    DATA_bin=DATA_bin.set_index("Rating class")
    return DATA_bin

In [59]:
# Funkcja przyjmuje tabelę, która w której są dane podziału na N grup, w każdej grupie znana jest liczba przypadków oraz probability of default

def Norm_Test(DF):
    DATA_bin=pd.DataFrame(columns=["5.0%","95.0%","1.0%","99.0%","0.01%","99.9%"])
    def f(q,N,p,k):
        return (p+k*norm.ppf((q+1)/2)*math.sqrt((p*(1-p))/N))
    for i in range(len(DF)):
        N_temp=DF["Total Number"][i+1]
        prob_tym=DF["Probability of default"][i+1]
        DATA_bin.loc[len(DATA_bin)]=[f(0.95,N_temp,prob_tym,-1),f(0.95,N_temp,prob_tym,1),f(0.98,N_temp,prob_tym,-1),f(0.98,N_temp,prob_tym,1),f(0.998,N_temp,prob_tym,-1),f(0.998,N_temp,prob_tym,1)]
    lista=list(range(1,len(DF)+1))
    DATA_bin.insert(0,"Rating class", lista)
    DATA_bin=DATA_bin.set_index("Rating class")
    return DATA_bin

In [60]:
Data2=Data1.loc[[1,2,3,4,5,6,7,8,9,10]]
Data2

,Number of Goods,Number of Bads,Total Number,Realized default rate,Probability of default,HL stat,Brier Score
Rating class,,,,,,,
1,58.0,0.0,58.0,0.0000,0.0000,0.0000,0.0000
2,58.0,0.0,58.0,0.0000,0.0000,0.0000,0.0000
3,58.0,0.0,58.0,0.0000,0.0001,0.0067,0.0000
4,58.0,0.0,58.0,0.0000,0.0006,0.0332,0.0000
5,58.0,0.0,58.0,0.0000,0.0020,0.1134,0.0002
6,57.0,0.0,57.0,0.0000,0.0166,0.9631,0.0157
7,57.0,0.0,57.0,0.0000,0.0692,4.2401,0.2732
8,52.0,5.0,57.0,0.0877,0.2911,11.4278,6.9198
9,31.0,26.0,57.0,0.4561,0.7716,32.1780,19.8117


In [61]:
Binomial_Test(Data2)

,5.0%,95.0%,1.0%,99.0%,0.01%,99.9%
Rating class,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241
4,0.000000,0.017241,0.000000,0.017241,0.000000,0.017241
5,0.000000,0.017241,0.000000,0.017241,0.000000,0.034483
6,0.000000,0.052632,0.000000,0.070175,0.000000,0.087719
7,0.017544,0.140351,0.000000,0.157895,0.000000,0.192982
8,0.175439,0.403509,0.157895,0.438596,0.122807,0.491228
9,0.666667,0.877193,0.631579,0.894737,0.596491,0.929825


In [62]:
Norm_Test(Data2)

,5.0%,95.0%,1.0%,99.0%,0.01%,99.9%
Rating class,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.002473,0.002673,-0.002954,0.003154,-0.003957,0.004157
4,-0.005702,0.006902,-0.006880,0.008080,-0.009336,0.010536
5,-0.009498,0.013498,-0.011647,0.015647,-0.016128,0.020128
6,-0.016569,0.049769,-0.022769,0.055969,-0.035697,0.068897
7,0.003314,0.135086,-0.009002,0.147402,-0.034681,0.173081
8,0.173170,0.409030,0.151125,0.431075,0.105162,0.477038
9,0.662618,0.880582,0.642246,0.900954,0.599770,0.943430


TRAFFIC LIGHTS APPROACH

In [25]:
def Q(DF,cor):
    Q=pd.DataFrame(columns=["95.0%","99.0%","99.9%"])
    def f(q,c,p):
        return norm.cdf((math.sqrt(c)*norm.ppf(q)+norm.ppf(p))/math.sqrt(1-c))
    for i in range(len(DF)):
        p=DF["Probability of default"].iloc[i]
        Q.loc[len(Q)]=[f(0.95,cor,p),f(0.98,cor,p),f(0.998,cor,p)]
    lista=list(range(1,11))
    lista.append("All classes")
    Q.insert(0,"Rating class", lista)
    Q=Q.set_index("Rating class")
    return Q

In [26]:
Q=Q(Data1,0.1)
Q

,95.0%,99.0%,99.9%
Rating class,,,
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000728,0.001156,0.002784
4,0.002407,0.003650,0.008019
5,0.007368,0.010654,0.021296
6,0.032545,0.043787,0.075853
7,0.127110,0.157723,0.232983
8,0.363661,0.415901,0.524883
9,0.805198,0.840529,0.898214


In [27]:
def Upper_Limit(DF,Q,cor):
    Upper=pd.DataFrame(columns=["95.0%","99.0%","99.9%"])
    def f(Q,N,q,p,c):
        return Q + (1/(2*N))*(2*Q-1 + ((Q*(1-Q))/(norm.cdf((math.sqrt(c)*norm.ppf(1-q)-norm.ppf(p))/(math.sqrt(1-c)))))*((math.sqrt(c)*(1-q)-norm.ppf(p))/(math.sqrt(1-c))-math.sqrt((1-c)/c)*norm.ppf(1-q)))
    for i in range(len(DF)):
        p=DF["Probability of default"].iloc[i]
        N=DF["Total Number"].iloc[i]
        Q1=Q["95.0%"].iloc[i]
        Q2=Q["99.0%"].iloc[i]
        Q3=Q["99.9%"].iloc[i]
        Upper.loc[len(Upper)]=[f(Q1,N,0.95,p,cor),f(Q3,N,0.98,p,cor),f(Q3,N,0.998,p,cor)]
    lista=list(range(1,11))
    lista.append("All classes")
    Upper.insert(0,"Rating class", lista)
    Upper=Upper.set_index("Rating class")
    return Upper
        

EXPERTS PD MODEL

In [66]:
expert_weights = [.2, .1, .1, .15, .25, .2]
expert_scores = ['PRODUCT_DEMAND', 'OWNERS_MANAGEMENT', 'ACCESS_CREDIT', 'PROFITABILITY', 'SHORT_TERM_LIQUIDITY', 'MEDIUM_TERM_LIQUIDITY']

In [85]:
df1 = df
df1['Score'] = np.dot(df[expert_scores], expert_weights)
df1 = df1[['DEFAULT_FLAG', 'Score']]
df1

,DEFAULT_FLAG,Score
0,0.0,51.10
1,0.0,80.00
2,0.0,44.00
3,0.0,63.10
4,0.0,56.95
...,...,...
5796,0.0,60.20
5797,0.0,57.75
5798,0.0,65.85
5799,0.0,57.85


In [96]:
df1['expert_PD'] = 1/(1+np.exp(-0.1*df1['Score']))

/tmp/ipykernel_49784/3720735218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['expert_PD'] = 1/(1+np.exp(-0.1*df1['Score']))


,DEFAULT_FLAG,Score,PD,expert_PD
0,0.0,51.10,0.994000,0.994000
1,0.0,80.00,0.999665,0.999665
2,0.0,44.00,0.987872,0.987872
3,0.0,63.10,0.998185,0.998185
4,0.0,56.95,0.996649,0.996649
...,...,...,...,...
5796,0.0,60.20,0.997576,0.997576
5797,0.0,57.75,0.996905,0.996905
5798,0.0,65.85,0.998621,0.998621
5799,0.0,57.85,0.996936,0.996936


In [118]:
min(df1['expert_PD']), max(df1['expert_PD'])

(0.7310585786300049, 0.9998766054240137)

In [140]:
expert_threshold = (max(df1['expert_PD']) + min(df1['expert_PD']))/2
expert_threshold

0.8654675920270093

In [115]:
Regr_PD = model.predict_proba(X_final)[:,1]
df1['Regr_PD'] = Regr_PD
df1

/tmp/ipykernel_49784/3139169079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Regr_PD'] = Regr_PD


,DEFAULT_FLAG,Score,PD,expert_PD,Regr_PD
0,0.0,51.10,0.994000,0.994000,1.224160e-01
1,0.0,80.00,0.999665,0.999665,1.044181e-18
2,0.0,44.00,0.987872,0.987872,9.422757e-01
3,0.0,63.10,0.998185,0.998185,1.770812e-04
4,0.0,56.95,0.996649,0.996649,1.209730e-02
...,...,...,...,...,...
5796,0.0,60.20,0.997576,0.997576,7.501420e-04
5797,0.0,57.75,0.996905,0.996905,1.021703e-03
5798,0.0,65.85,0.998621,0.998621,2.825078e-06
5799,0.0,57.85,0.996936,0.996936,1.667666e-03


In [119]:
min(df1['Regr_PD']), max(df1['Regr_PD'])

(1.27439100213817e-24, 0.9999805941781736)

In [141]:
df1['expert_flag'] = df1['expert_PD'] < expert_threshold 
### UWAGA: w tym modelu ekspertów jest chyb a taki błąd, że on liczy 1-PD a nie PD, dlatego powyżej nierówność <
df1['Regr_flag'] = df1['Regr_PD']>0.5
df1

/tmp/ipykernel_49784/2914062253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['expert_flag'] = df1['expert_PD'] < expert_threshold
/tmp/ipykernel_49784/2914062253.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Regr_flag'] = df1['Regr_PD']>0.5


,DEFAULT_FLAG,Score,PD,expert_PD,Regr_PD,expert_flag,Regr_flag
0,0.0,51.10,0.994000,0.994000,1.224160e-01,False,False
1,0.0,80.00,0.999665,0.999665,1.044181e-18,False,False
2,0.0,44.00,0.987872,0.987872,9.422757e-01,False,True
3,0.0,63.10,0.998185,0.998185,1.770812e-04,False,False
4,0.0,56.95,0.996649,0.996649,1.209730e-02,False,False
...,...,...,...,...,...,...,...
5796,0.0,60.20,0.997576,0.997576,7.501420e-04,False,False
5797,0.0,57.75,0.996905,0.996905,1.021703e-03,False,False
5798,0.0,65.85,0.998621,0.998621,2.825078e-06,False,False
5799,0.0,57.85,0.996936,0.996936,1.667666e-03,False,False


In [142]:
#sum(df1['DEFAULT_FLAG'] != df1['expert_flag'])/len(df1) # model ekspertów 
confusion_matrix(df1['DEFAULT_FLAG'],df1['expert_flag'])

array([[5210,    3],
       [ 520,    8]])

In [143]:
#sum(df1['DEFAULT_FLAG'] != df1['Regr_flag'])/len(df1) # nasz 
confusion_matrix(df1['DEFAULT_FLAG'],df1['Regr_flag'])

array([[4642,  571],
       [  11,  517]])